In [ ]:
# ml/build_set_features_from_frame_logs.py

import os
import glob
import numpy as np
import pandas as pd


# ==============================
# 1. 설정
# ==============================

# frame 로그들이 있는 폴더
DATA_DIR = "."

# frame 로그 파일 패턴
FRAME_LOG_PATTERN = "frame_logs_right_curl*.csv"  # 필요하면 수정

# 출력 파일 이름 (여러 파일 합쳐서 한 방에 만들기)
OUTPUT_PATH = "set_features_with_rule_scores.csv"


# ==============================
# 2. 룰 기반 점수 계산 함수
# ==============================
def compute_rule_score(
    rep_count: int,
    mean_rom: float,
    std_rom: float,
    mean_tempo: float,
    bad_ratio: float,
) -> float:
    """
    세트 요약 지표들을 가지고 1.0 ~ 5.0 사이의 rule_score를 계산.
    (점수 설계는 필요하면 자유롭게 튜닝하면 된다.)
    """
    score = 5.0

    # 1) 반복 수 페널티 (예: 10회 정도를 이상적인 세트로 가정)
    if rep_count < 5:
        score -= 1.5
    elif rep_count < 8:
        score -= 1.0
    elif rep_count < 10:
        score -= 0.5
    elif rep_count > 15:
        score -= 0.5

    # 2) ROM (각도 차이) 페널티
    #   - 예: 평균 ROM 80도 이상이면 좋음, 60 이하면 페널티
    if mean_rom < 40:
        score -= 2.0
    elif mean_rom < 60:
        score -= 1.0

    # ROM 일관성 (std_rom 너무 크면 반복마다 들쭉날쭉)
    if std_rom > 20:
        score -= 0.5

    # 3) 템포 (rep 당 걸리는 시간)
    #   - 예: 0.8~3초 사이가 적당하다고 가정
    if mean_tempo < 0.6:
        score -= 0.5  # 너무 빠름
    elif mean_tempo > 4.0:
        score -= 0.5  # 너무 느림

    # 4) 나쁜 프레임 비율 (BAD_ELBOW 등)
    #   - 프레임 중 'GOOD'이 아닌 비율이 클수록 큰 페널티
    score -= 3.0 * bad_ratio  # bad_ratio 0.5면 1.5점 깎임

    # 점수 범위 클램핑
    score = max(1.0, min(5.0, score))
    return float(score)


# ==============================
# 3. 세트 피처 생성
# ==============================
def build_set_features_from_frame_logs(df: pd.DataFrame) -> pd.DataFrame:
    """
    frame 단위 로그 DataFrame으로부터
    세트 단위 피처 + rule_score DataFrame을 생성한다.
    """
    required_cols = ["set_id", "rep_id", "ts", "elbow_angle", "label"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"⚠️ frame_logs DataFrame에 아래 컬럼이 없습니다: {missing}")

    # 타입 정리 (ts, elbow_angle은 float 형으로 맞춰준다)
    df = df.copy()
    df["ts"] = df["ts"].astype(float)
    df["elbow_angle"] = df["elbow_angle"].astype(float)

    # 세트별로 그룹핑 (파일 구분용 session_name이 있으면 같이 쓰면 더 좋음)
    group_cols = ["set_id"]
    if "session_name" in df.columns:
        group_cols = ["session_name", "set_id"]

    set_rows = []

    for set_key, g in df.groupby(group_cols):
        # set_key: set_id 또는 (session_name, set_id)
        if isinstance(set_key, tuple):
            key_dict = {
                "session_name": set_key[0],
                "set_id": set_key[1],
            }
        else:
            key_dict = {"set_id": set_key}

        # ---------------------------
        # 1) 기본 통계
        # ---------------------------
        rep_count = g["rep_id"].nunique()
        min_elbow = g["elbow_angle"].min()
        max_elbow = g["elbow_angle"].max()
        mean_elbow = g["elbow_angle"].mean()
        std_elbow = g["elbow_angle"].std(ddof=0)  # 모표준편차 (편한 쪽 쓰면 됨)

        # ---------------------------
        # 2) rep 단위 ROM, tempo 계산
        # ---------------------------
        rom_list = []
        tempo_list = []

        for rep_id, gr in g.groupby("rep_id"):
            rom = gr["elbow_angle"].max() - gr["elbow_angle"].min()
            tempo = gr["ts"].max() - gr["ts"].min()

            rom_list.append(rom)
            tempo_list.append(tempo)

        if len(rom_list) > 0:
            mean_rom = float(np.mean(rom_list))
            std_rom = float(np.std(rom_list))
        else:
            mean_rom, std_rom = 0.0, 0.0

        if len(tempo_list) > 0:
            mean_tempo = float(np.mean(tempo_list))
            std_tempo = float(np.std(tempo_list))
        else:
            mean_tempo, std_tempo = 0.0, 0.0

        # ---------------------------
        # 3) BAD 프레임 비율
        # ---------------------------
        total_frames = len(g)
        bad_frames = (g["label"] != "GOOD").sum()
        bad_ratio = bad_frames / total_frames if total_frames > 0 else 0.0

        # ---------------------------
        # 4) rule_score 계산
        # ---------------------------
        rule_score = compute_rule_score(
            rep_count=rep_count,
            mean_rom=mean_rom,
            std_rom=std_rom,
            mean_tempo=mean_tempo,
            bad_ratio=bad_ratio,
        )

        row = {
            **key_dict,
            "rep_count": rep_count,
            "min_elbow_angle": float(min_elbow),
            "max_elbow_angle": float(max_elbow),
            "mean_elbow_angle": float(mean_elbow),
            "std_elbow_angle": float(std_elbow if not np.isnan(std_elbow) else 0.0),
            "mean_rom": mean_rom,
            "std_rom": std_rom,
            "mean_tempo": mean_tempo,
            "std_tempo": std_tempo,
            "bad_frame_ratio": float(bad_ratio),
            "rule_score": rule_score,
        }

        set_rows.append(row)

    set_df = pd.DataFrame(set_rows)
    return set_df


# ==============================
# 4. main: frame_logs_* → set_features_with_rule_scores.csv
# ==============================
def main():
    search_pattern = os.path.join(DATA_DIR, FRAME_LOG_PATTERN)
    frame_files = sorted(glob.glob(search_pattern))

    print("===================================")
    print(f"[INFO] frame 로그 검색 패턴: {search_pattern}")
    print(f"[INFO] 발견한 frame_logs 파일 수: {len(frame_files)}")
    for f in frame_files:
        print(f"  - {f}")
    print("===================================")

    if len(frame_files) == 0:
        raise RuntimeError("⚠️ frame_logs CSV 파일을 하나도 찾지 못했습니다.")

    all_frames = []

    for path in frame_files:
        try:
            df = pd.read_csv(path)
            # 세션 구분용 컬럼 추가 (원하면 나중에 드롭해도 됨)
            session_name = os.path.splitext(os.path.basename(path))[0]
            df["session_name"] = session_name
            all_frames.append(df)
        except Exception as e:
            print(f"[WARN] 파일 읽기 실패: {path} ({e})")

    if len(all_frames) == 0:
        raise RuntimeError("⚠️ CSV 읽기 실패로 인해 유효한 frame_logs DataFrame이 없습니다.")

    frame_df = pd.concat(all_frames, ignore_index=True)
    print(f"[INFO] frame_logs 전체 크기: {frame_df.shape}")

    set_df = build_set_features_from_frame_logs(frame_df)
    print(f"[INFO] 세트 피처 DataFrame 크기: {set_df.shape}")

    set_df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
    print(f"[INFO] 세트 피처 + rule_score CSV 저장 완료: {OUTPUT_PATH}")


if __name__ == "__main__":
    main()
